In [80]:
import torch
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
import random

# Load names from SortedLowercase.txt
with open('Data/SortedLowercase.txt', 'r') as file:
    names = file.read().splitlines()

# Build a character-to-index and index-to-character dictionary
chars = sorted(list(set(''.join(names))))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# Convert names into sequences of indices
def name_to_indices(name):
    return [char_to_idx[char] for char in name]

sequences = [name_to_indices(name) for name in names]
vocab_size = len(chars)

In [81]:
print(names)
print(chars)

['աբգար', 'աբեթ', 'աբել', 'աբիկ', 'աբիկ', 'աբով', 'աբրահամ', 'աբրամ', 'ագաթա', 'ագապէ', 'ագապի', 'ագափէ', 'ագնէս', 'ադա', 'ադամ', 'ադան', 'ադդէ', 'ադելայիդա', 'ադելինա', 'ադէ', 'ադիկ', 'ադիս', 'ադոն', 'ադրիանա', 'ադրիկ', 'ադրիկ', 'ադրինէ', 'ադօ', 'ազատ', 'ազատուհի', 'ազարիա', 'ազարիկ', 'ազգանուշ', 'ազգեր', 'ազգուշ', 'ազիկ', 'ազնաւոր', 'ազնաւուր', 'ազնի', 'ազնիկ', 'ազնիւ', 'ազնուր', 'ազնւիկ', 'ազօ', 'աթանիա', 'աթանիա', 'աթինէ', 'աթրիս', 'ալան', 'ալան', 'ալարիկ', 'ալբերտ', 'ալբերթ', 'ալբրիկ', 'ալենուշ', 'ալեքսան', 'ալեքսանդր', 'ալէն', 'ալէք', 'ալէքս', 'ալիծ', 'ալին', 'ալինա', 'ալիս', 'ալիսա', 'ալլա', 'ալմա', 'ալմաս', 'ալմարա', 'ալմիկ', 'ալվարդ', 'ալւա', 'ալւանդ', 'ալւինա', 'ալօ', 'ալֆրէդ', 'ախթամար', 'ախուրա', 'ախուրի', 'ախուրիկ', 'ականց', 'ակեր', 'ակէ', 'ակներ', 'ակնէ', 'ակնէ', 'ակնի', 'ակնիկ', 'ակնունի', 'ակոբ', 'ահարոն', 'աղան', 'աղասի', 'աղաւնի', 'աղաւնիկ', 'աղբիւրակ', 'աղեքսանդր', 'աղուշ', 'աղւոր', 'աղօթքից', 'ամալիա', 'ամալիկ', 'ամանդա', 'ամաս', 'ամասիա', 'ամատունի', 'ամարաս', 'ամբ

In [82]:
chars.append('<pad>')  # Add padding token to vocabulary
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}
pad_idx = char_to_idx['<pad>']  # Use the correct index for the padding token
vocab_size = len(chars)  # Update vocab size to include the padding token


In [83]:
from torch.nn.utils.rnn import pad_sequence

class NameDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        input_seq = torch.tensor(sequence[:-1], dtype=torch.long)  # All chars except last
        target_seq = torch.tensor(sequence[1:], dtype=torch.long)  # All chars except first
        return input_seq, target_seq

# Collate function to pad sequences in each batch
def collate_fn(batch):
    input_seqs, target_seqs = zip(*batch)
    input_seqs_padded = pad_sequence(input_seqs, batch_first=True, padding_value=pad_idx)
    target_seqs_padded = pad_sequence(target_seqs, batch_first=True, padding_value=pad_idx)
    return input_seqs_padded, target_seqs_padded


dataset = NameDataset(sequences)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [84]:
import torch.nn as nn

class NameGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128, pad_idx=None):
        super(NameGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

In [85]:
import torch.optim as optim

model = NameGenerator(vocab_size=vocab_size, pad_idx=pad_idx)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
for epoch in range(num_epochs):
    total_loss = 0
    for input_seq, target_seq in dataloader:
        output, _ = model(input_seq)
        output = output.view(-1, vocab_size)
        target_seq = target_seq.view(-1)

        loss = criterion(output, target_seq)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")


Epoch [1/50], Loss: 2.3826
Epoch [2/50], Loss: 1.5514
Epoch [3/50], Loss: 1.3935
Epoch [4/50], Loss: 1.3360
Epoch [5/50], Loss: 1.2881
Epoch [6/50], Loss: 1.2457
Epoch [7/50], Loss: 1.2060
Epoch [8/50], Loss: 1.2252
Epoch [9/50], Loss: 1.1867
Epoch [10/50], Loss: 1.1533
Epoch [11/50], Loss: 1.1578
Epoch [12/50], Loss: 1.1117
Epoch [13/50], Loss: 1.1034
Epoch [14/50], Loss: 1.0624
Epoch [15/50], Loss: 1.0668
Epoch [16/50], Loss: 1.0489
Epoch [17/50], Loss: 1.0200
Epoch [18/50], Loss: 1.0018
Epoch [19/50], Loss: 0.9759
Epoch [20/50], Loss: 0.9668
Epoch [21/50], Loss: 0.9516
Epoch [22/50], Loss: 0.9387
Epoch [23/50], Loss: 0.9153
Epoch [24/50], Loss: 0.9013
Epoch [25/50], Loss: 0.8994
Epoch [26/50], Loss: 0.8778
Epoch [27/50], Loss: 0.8713
Epoch [28/50], Loss: 0.8543
Epoch [29/50], Loss: 0.8374
Epoch [30/50], Loss: 0.8277
Epoch [31/50], Loss: 0.8283
Epoch [32/50], Loss: 0.8054
Epoch [33/50], Loss: 0.7973
Epoch [34/50], Loss: 0.7898
Epoch [35/50], Loss: 0.7843
Epoch [36/50], Loss: 0.7758
E

In [90]:
def generate_names(model, num_names=10, max_length=10):
    model.eval()
    generated_names = []
    
    for _ in range(num_names):
        # Start with a random character
        start_char = random.choice(list(char_to_idx.keys()))
        indices = [char_to_idx[start_char]]
        length = random.randrange(3,max_length)
        # Generate the name
        for _ in range(length - 1):
            input_seq = torch.tensor([indices], dtype=torch.long)
            with torch.no_grad():
                output, _ = model(input_seq)
                
            # Sample from the output distribution to get the next character index
            next_char_idx = torch.argmax(output[0, -1]).item()
            indices.append(next_char_idx)
            
            # Stop if we generate a padding character or repeat starting char
            if idx_to_char[next_char_idx] == '<pad>':
                break
        
        # Convert indices to characters and append the generated name
        generated_name = ''.join(idx_to_char[idx] for idx in indices if idx in idx_to_char)
        generated_names.append(generated_name)
    
    return generated_names

# Example usage:
print(generate_names(model, num_names=50))


['օսէփիկ', 'ռուստամա', 'ճարտարատա', 'երանո', 'բարթուրի', 'նորազնա', 'ցոլինէսի', 'թամար', 'ُ<pad>', 'ծաղկանուշ', 'ղազարուհի', 'նորազնակա', 'ձօն', 'պարսամա', 'ձօնիկա', 'գարունի', 'ժան', 'ոսկեհատա', 'լուսա', 'յովհաննէս', 'գարունիկ', 'նորազնակ', 'մարի', 'ոսկեհ', 'քաջազնու', 'սար', 'բարթու', 'զարմա', 'ُ<pad>', 'արամայիսի', 'ոսկ', 'օսէ', 'ժանէ', 'արամայիսի', 'ծաղկա', 'ծաղկանու', 'ցոլ', 'յովհ', 'լուսանու', 'արամայիս', 'երանո', 'ժանէթէլ', 'ջոջիկանա', 'ռուս', 'պար', 'դաւթիկ', 'ոսկեհա', 'շահականո', 'թամարական', 'րբակու']
